**Lecture 3**

Estimate alphas and betas using market model regressions



**Data Description**

Input Dataframes

1.  "asset_returns" dataframe : Contains monthly returns for assets for which market model parameters are to be estimated.

input file: 'b2m_monthlyReturns.csv'


2.   "Mkt_Risk_freeRate " dataframe : contains monthly market returns and riskfree rate

Market = Market-cap weighted index of all US stocks from CRSP (vwretd)

input file: 'Market_Riskfree.csv'


Output: Alpha and beta estimates and t-statistics the for assets in  "asset_returns" dataframe







In [1]:
# Connecting the Python Code with the google drive to access the datasets
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Importing Necessary Python Libraries
import pandas as pd
import statsmodels.api as sm

In [2]:
# Edit the path to your data here (ensure your dataset has all variables)
returns_data_path = '/content/drive/MyDrive/MAF data/b2m_monthlyReturns.csv'
Mkt_Risk_freeRate_path = '/content/drive/MyDrive/MAF data/Market_Riskfree_2312.csv'

In [4]:
# Function to load/preprocess data
def load_data(returns_data_path, Mkt_Risk_freeRate_path):
    asset_returns = pd.read_csv(returns_data_path)
    Mkt_Risk_freeRate = pd.read_csv(Mkt_Risk_freeRate_path)
    Mkt_Risk_freeRate['mkt_excess_returns'] = Mkt_Risk_freeRate['Market'] - Mkt_Risk_freeRate['RiskfreeRate']
    asset_returns = pd.merge(asset_returns, Mkt_Risk_freeRate, how = "inner", on = ['year', 'month'])
    asset_returns['excess_returns'] = asset_returns['RET'] - asset_returns['RiskfreeRate']
    asset_returns.loc[asset_returns['rank'] == 'diff',  'excess_returns'] = asset_returns['RET']
    return asset_returns

# Store your data in asset_returns dataframe
asset_returns = load_data(returns_data_path,Mkt_Risk_freeRate_path)
asset_returns.head(11)

,year,month,rank,RET,Market,RiskfreeRate,mkt_excess_returns,excess_returns
0,1990,11,0,0.060835,0.0692,0.0057,0.0635,0.055135
1,1990,11,1,0.098999,0.0692,0.0057,0.0635,0.093299
2,1990,11,2,-0.094637,0.0692,0.0057,0.0635,-0.100337
3,1990,11,3,0.092174,0.0692,0.0057,0.0635,0.086474
4,1990,11,4,-0.011416,0.0692,0.0057,0.0635,-0.017116
5,1990,11,5,0.013274,0.0692,0.0057,0.0635,0.007574
6,1990,11,6,-0.007936,0.0692,0.0057,0.0635,-0.013636
7,1990,11,7,0.140500,0.0692,0.0057,0.0635,0.134800
8,1990,11,8,0.017750,0.0692,0.0057,0.0635,0.012050
9,1990,11,9,-0.060708,0.0692,0.0057,0.0635,-0.066408


In [5]:
# Function to get ranks list
def get_ranks_list(asset_returns):
    ranks = set((asset_returns['rank']))
    return list(sorted(ranks))

# Print ranks list (just to confirm)
ranks_list = get_ranks_list(asset_returns)
ranks_list

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'diff']

In [6]:
# Function to estimate alphas and betas
def estimate_alphas_betas(asset_returns, ranks_list):
    asset_returns['constant'] = 1
    for decile in ranks_list:
        x = asset_returns[['constant', 'mkt_excess_returns']].loc[asset_returns['rank'] == decile].copy()
        y = asset_returns['excess_returns'].loc[asset_returns['rank'] == decile].copy()
        model = sm.OLS(y, x)
        results = model.fit()
        print('************************************************')
        print('Decile:' , decile)
        print('**************** estimates of alpha and beta ****************' )
        print(results.params)
        print('******** t-statistics for alpha and beta ***********' )
        print(results.tvalues)

# Estimate alphas and betas
estimate_alphas_betas(asset_returns, ranks_list)

************************************************
Decile: 0
**************** estimates of alpha and beta ****************
constant              0.000473
mkt_excess_returns    1.424972
dtype: float64
******** t-statistics for alpha and beta ***********
constant               0.166176
mkt_excess_returns    22.372871
dtype: float64
************************************************
Decile: 1
**************** estimates of alpha and beta ****************
constant             -0.000421
mkt_excess_returns    1.187813
dtype: float64
******** t-statistics for alpha and beta ***********
constant              -0.230837
mkt_excess_returns    29.100376
dtype: float64
************************************************
Decile: 2
**************** estimates of alpha and beta ****************
constant             -0.001165
mkt_excess_returns    1.064662
dtype: float64
******** t-statistics for alpha and beta ***********
constant              -0.764832
mkt_excess_returns    31.254894
dtype: float64
**********